In [ ]:
import os
import pandas as pd

from polygon import RESTClient
from datetime import datetime, timedelta, timezone


# --- Config ---
API_KEY = os.getenv("POLYGON_API_KEY")  # make sure this is set
PAIR = "C:EURUSD"  # FX ticker format
base_dir = "../data/agg_data/fx/"
os.makedirs(base_dir, exist_ok=True)

# --- Date range: last 1 year ---
end_date = datetime.now(timezone.utc).date()
start_date = end_date - timedelta(days=365)

# --- Initialize Polygon client ---
client = RESTClient(api_key=API_KEY)

# --- Fetch aggregates (pagination handled automatically) ---
print(f"⏳ Downloading 1y of {PAIR} 1m aggregates via Polygon client...")
aggs = []
for bar in client.list_aggs(
    ticker=PAIR,
    multiplier=1,
    timespan="minute",
    from_=str(start_date),
    to=str(end_date),
    adjusted=True,
    limit=50000,  # page size; client auto-paginates
):
    aggs.append(bar)

print(f"✅ Retrieved {len(aggs)} bars")

# --- Convert to DataFrame ---
df = pd.DataFrame([a.__dict__ for a in aggs])

# Convert timestamp
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

# Save to CSV
out_path = os.path.join(base_dir, f"{PAIR}_1m_last1y.csv")
df.to_csv(out_path, index=False)

print(f"✅ Saved {len(df)} rows to {out_path}")


/var/folders/by/24jm3kg56bqc__yntqtthmyw0000gn/T/ipykernel_10690/2886748142.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_date = datetime.utcnow().date()


⏳ Downloading 1y of C:EURUSD 1m aggregates via Polygon client...
✅ Retrieved 368307 bars
✅ Saved 368307 rows to agg_data/fx/C:EURUSD_1m_last1y.csv
